In [1]:
import tensorflow as tf

In [2]:
u = tf.random.normal(shape=[2,3,5])
v = tf.random.normal(shape=[2,3,5])

In [3]:
u

<tf.Tensor: shape=(2, 3, 5), dtype=float32, numpy=
array([[[ 0.31770432, -0.09981439, -1.3909899 , -1.4619008 ,
          0.8406374 ],
        [-0.31458592, -0.12878741,  0.6743781 , -0.02826239,
          0.6999799 ],
        [ 0.20014113,  0.57743347, -0.87986815,  0.6497242 ,
         -0.90177023]],

       [[-1.066647  ,  0.5749734 , -0.2209456 ,  0.12389447,
          0.14851066],
        [ 0.24649505,  1.7933253 ,  0.03395426, -0.17940024,
          1.0305643 ],
        [ 1.2068981 ,  1.1124333 , -1.1829511 ,  0.69570845,
         -0.6409244 ]]], dtype=float32)>

In [4]:
v

<tf.Tensor: shape=(2, 3, 5), dtype=float32, numpy=
array([[[-0.90792966,  2.0739152 , -1.3732276 , -1.6966985 ,
         -0.75434935],
        [-0.13587275,  0.6264506 ,  1.6584879 ,  0.4990391 ,
          0.27517536],
        [-0.58823174,  1.7257879 , -1.8042158 ,  0.92529625,
          1.0974452 ]],

       [[-0.9484317 ,  1.2036116 , -0.77518165, -0.7529136 ,
         -0.525763  ],
        [-1.5086113 , -0.8617532 , -0.08409377,  2.6180634 ,
         -0.3102877 ],
        [ 0.09926008, -0.809405  , -1.4375613 , -0.4598477 ,
         -0.5357119 ]]], dtype=float32)>

In [5]:
v_t = tf.einsum('mij->mji', v) 
v_dag = tf.math.conj(v_t)
v_dag

<tf.Tensor: shape=(2, 5, 3), dtype=float32, numpy=
array([[[-0.90792966, -0.13587275, -0.58823174],
        [ 2.0739152 ,  0.6264506 ,  1.7257879 ],
        [-1.3732276 ,  1.6584879 , -1.8042158 ],
        [-1.6966985 ,  0.4990391 ,  0.92529625],
        [-0.75434935,  0.27517536,  1.0974452 ]],

       [[-0.9484317 , -1.5086113 ,  0.09926008],
        [ 1.2036116 , -0.8617532 , -0.809405  ],
        [-0.77518165, -0.08409377, -1.4375613 ],
        [-0.7529136 ,  2.6180634 , -0.4598477 ],
        [-0.525763  , -0.3102877 , -0.5357119 ]]], dtype=float32)>

In [6]:
prod = tf.einsum('mij, mjk -> mik', v_dag, u)
prod

<tf.Tensor: shape=(2, 5, 5), dtype=float32, numpy=
array([[[-0.36343887, -0.23154154,  1.6888578 ,  0.9489549 ,
         -0.32789797],
        [ 0.80722046,  0.70884216, -3.9807963 , -1.9282773 ,
          0.62564945],
        [-1.3191152 , -1.1183391 ,  4.6160655 ,  0.7884071 ,
          1.6335099 ],
        [-0.51084924,  0.639382  ,  1.8824928 ,  3.0674882 ,
         -1.9113955 ],
        [-0.10658242,  0.6735574 ,  0.2692575 ,  1.8080435 ,
         -1.4311604 ]],

       [[ 0.7595734 , -3.1403337 ,  0.0409082 ,  0.22219586,
         -1.7591914 ],
        [-2.473116  , -1.7537682 ,  0.6622937 , -0.25939038,
         -0.19057554],
        [-0.9288735 , -2.1957073 ,  1.8689823 , -1.0810778 ,
          0.71958125],
        [ 0.8934434 ,  3.7505841 ,  0.7992247 , -0.88288295,
          2.8809946 ],
        [-0.16223049, -1.4546902 ,  0.7393504 , -0.38217273,
         -0.05450201]]], dtype=float32)>

In [7]:
prod_tr = tf.einsum('mii->m', prod)
prod_tr

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 6.597797  , -0.06259745], dtype=float32)>

In [14]:
prod_div = prod_tr/2
prod_div

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 3.2988985 , -0.03129873], dtype=float32)>

In [20]:
prod_tr_dag= tf.math.conj(prod_tr)
tf.einsum('i,i->i', prod_tr_dag, prod_tr)


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([4.3530926e+01, 3.9184410e-03], dtype=float32)>

In [26]:
prod_div - 1

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 2.2988985, -1.0312988], dtype=float32)>

In [25]:
tf.math.square(prod_div)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.0882731e+01, 9.7961025e-04], dtype=float32)>

In [24]:
c = tf.constant(3.2988985)
c**2

<tf.Tensor: shape=(), dtype=float32, numpy=10.882731>

In [12]:
m = tf.reduce_mean(prod_div, axis = 0)
m

<tf.Tensor: shape=(), dtype=float32, numpy=1.6337999>

In [9]:
abstf.math.abs(prod_tr)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([6.597797  , 0.06259745], dtype=float32)>

In [13]:
tf.squeeze(m)

<tf.Tensor: shape=(), dtype=float32, numpy=1.6337999>

In [26]:
def fid():
    '''
    return cost function 1- |tr(U_f^dag * U(T)) / D|^2 where U_f is target unitary and U(T)/U_tot 
    unitary function of parameters
    
    einsum notation: tensorflow.org/api_docs/python/tf/einsum
    '''
   #dimension 
    #d = self.parameters["N_cav1"] + self.parameters["N_cav2"] + 2 #2 for qubit
    D = 4#tf.constant(d, dtype=tf.complex64)

    #first comute U_tot
#     bs = self.batch_construct_block_operators(
#             betas_rho, betas_angle,gammas_rho, gammas_angle,
#             alphas1_rho, alphas1_angle, alphas2_rho, alphas2_angle,
#             phis, etas, thetas
#         )
    ## creating a M copies of identity where M is # of multistarts
    I = tf.cast(tf.eye(4), dtype = tf.complex128)# * self.parameters['N_cav1'] * self.parameters['N_cav2'])
    U_tot = tf.stack([I for i in range(3)]) #stacking for multistarts
#     for U in bs: 
#         U_tot = tf.einsum('mij, mjk -> mik', U, U_tot)   #m is multistart

    #Now compute tr(U_f^dag * U(T)) / D
    U_f = tf.stack([SWAP(2)for i in range(3)])
    U_f_dag = tf.math.conj(
                tf.einsum('mij-> mji', U_f)
    )
    prod = tf.einsum('mij, mjk -> mik', U_f_dag, U_tot)
    prod_tr = tf.einsum('mii->m', prod) / D   #trace and divison

    #now 1 - |...|^2
    fids =  tf.math.square(
                            tf.math.abs(prod_tr))
    return fids

    




In [27]:
fid()

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([0.25, 0.25, 0.25])>

In [ ]:
q

In [13]:
tf.eye(4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [16]:
#SWAP Operation
from qutip import *

def SWAP(N): 
    '''
    Returns swap unitary in qutip 
    '''
    rho = None
    N = 2 

    for i in range(N): 
        for j in range(N): 
            ket = tensor(basis(N, i), basis(N, j))
            bra = (tensor(basis(N, j), basis(N, i))).dag()
            rho_ = (1.0+0.0j)*ket * bra
            if rho==None: 
                rho = rho_
            else: 
                rho += rho_
    rho = tf.cast(rho, dtype = tf.complex128)
    return rho
            

In [17]:
SWAP(2)

<tf.Tensor: shape=(4, 4), dtype=complex128, numpy=
array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])>

In [44]:
u = tf.random.normal(shape=[2,3])
u

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.4120879 , 0.48583698, 0.8266079 ],
       [0.45285845, 0.7897659 , 0.111969  ]], dtype=float32)>

In [50]:
M = 2
tf.stack([u for i in range(M)])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[0.4120879 , 0.48583698, 0.8266079 ],
        [0.45285845, 0.7897659 , 0.111969  ]],

       [[0.4120879 , 0.48583698, 0.8266079 ],
        [0.45285845, 0.7897659 , 0.111969  ]]], dtype=float32)>